In [1]:
import numpy as np
import scipy
import scipy.optimize as opt
import scipy.sparse as sp

import matplotlib.pyplot as plt

In [9]:
N = int(1e2)
s = np.linspace(0, 1, N+1)
h = 1/N

vmax = 50.
g = 9.8
amin, amax = -2.*g, 4.*g
mu = 5.

x0, x1 = 12.5, -12.5
y0, y1 = 0., 0.
v0 = 17.5

In [10]:
D1 = sp.diags([-1, 0, 1], offsets=[0, 1, 2], shape=(N-1, N+1))
D2 = sp.diags([1, -2, 1], offsets=[0, 1, 2], shape=(N-1, N+1))

D1 = sp.vstack([sp.csr_array([-3, 4, -1]+[0]*(N-2)), D1, sp.csr_array([0]*(N-2)+[1, -4, 3])])
D2 = sp.vstack([sp.csr_array([2, -5, 4, -1]+[0]*(N-3)), D2, sp.csr_array([0]*(N-3) + [-1, 4, -5, 2])])

D1 = D1/(2*h)
D2 = D2/(h**2)

I = sp.diags([1, 1], offsets=[0, 1], shape=(N, N+1))
I = .5*h*I

In [11]:
x_init = 12.5*np.cos(np.pi*s)[1:-1]
y_init = 12.5*np.sin(np.pi*s)[1:-1]
v_init = v0*np.ones(N)

In [12]:
x_ = np.r_[x0, x_init, x1]
y_ = np.r_[y0, y_init, y1]
v_ = np.r_[v0, v_init]

D1x = D1@x_
D1y = D1@y_

d = (D1x**2 + D1y**2)**.5
t = np.sum(I@(d/v_))
aT = v_*(D1@v_)/d
aN = v_**2 * np.abs(D2@x_ * D1y - D2@y_*D1x)/d**3

In [26]:
def smooth_max(u, eps=1e-3):
    for i in range(len(u)):
        if u[i]<=0:
            u[i] = 0
        elif u[i]<=eps:
            u[i] = u[i]**2/(2*eps)
        else:
            u[i] = u[i] - eps/2
    return u

In [27]:
def f(x, y, v):
    x_ = np.r_[x0, x, x1]
    y_ = np.r_[y0, y, y1]
    v_ = np.r_[v0, v]

    D1x = D1@x_
    D1y = D1@y_

    d = (D1x**2 + D1y**2)**.5
    t = np.sum(I@(d/v_))
    aT = v_*(D1@v_)/d
    aN = v_**2 * np.abs(D2@x_ * D1y - D2@y_*D1x)/d**3
    r = (x**2 + y**2)**.5

    res = t
    res += smooth_max(-v)
    res += smooth_max(v - vmax)

    res += smooth_max(amin - aT)
    
    res += smooth_max(aT - amax)
    res += smooth_max(aN - mu*g)
    res += smooth_max(r - 15.)
    res += smooth_max(12.5 -r)
    return res

In [28]:
f(x_init, y_init, v_init)

ValueError: operands could not be broadcast together with shapes (100,) (101,) (100,) 